# Applied Data Science Capstone
## Final Project for the IBM Data Science Professional Certificate on Coursera
### Jonathan Kolb

This notebook is being used to create a machine learning model to predict Accident Severity based on various attributes.

In [4]:
#Import the pandas and Numpy libraries
import pandas as pd
import numpy as np

In [5]:
#Print the following the statement: Hello Capstone Project Course!
print ('Hello Capstone Project Course!')
#Personally I prefer the classic 'Hello World' instead

Hello Capstone Project Course!


## Week 3 Assignment
### Segmenting and Clustering Neighborhoods in Toronto
Thank you for reviewing my assignment! If you're reviewing this notebook for the week 3 assignment, please start here.

In [4]:
# I followed the guide located here: https://pythonbasics.org/pandas-web-scraping/
# I also followed the documentation for Beautiful Soup here: https://beautiful-soup-4.readthedocs.io/en/latest/
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
#We use pd.read_html to get the HTML content from the wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

In [11]:
#However, there are multiple lists on that URL. Luckily, the one we want is the first table.
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [14]:
#Next, We want to remove the records where the Borough is 'Not Assigned'
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
# The instructions say to set any Not Assigned neighbourhoods to match the Borough, but they don't seem to be there
df[df.Neighbourhood == 'Not Assigned']

,Postal Code,Borough,Neighbourhood


In [26]:
# Last Step for the assignment, display the shape.
df.shape

(103, 3)